<a href="https://colab.research.google.com/github/Mani512996/Data-Science-project/blob/main/Mainprojectfile1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance
!pip install plotly
!pip install chart_studio
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import sklearn
import seaborn as sns
from keras.models import Sequential
import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.offline import init_notebook_mode
import plotly.express as px
from plotly.subplots  import make_subplots
import chart_studio.plotly as py
from chart_studio.plotly import iplot
import plotly.graph_objects as go
from plotly.offline import iplot
from keras import optimizers
from datetime import date
import datetime as dt
from datetime import date

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 27.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in index

In [3]:
START = "2021-01-01"
TODAY = dt.datetime.now().strftime("%Y-%m-%d")




In [4]:
stock_symbol = 'ITC.NS'
df = pd.read_csv("ITC.NS.csv")

In [5]:
data = df.dropna()

In [6]:
len(data)


1236

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1236 entries, 0 to 1235
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1236 non-null   object 
 1   Open       1236 non-null   float64
 2   High       1236 non-null   float64
 3   Low        1236 non-null   float64
 4   Close      1236 non-null   float64
 5   Adj Close  1236 non-null   float64
 6   Volume     1236 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 77.2+ KB


In [8]:
data.describe

<bound method NDFrame.describe of             Date        Open        High         Low       Close   Adj Close  \
0     2017-07-25  295.000000  295.149994  290.850006  291.399994  242.203461   
1     2017-07-26  292.000000  294.250000  291.500000  293.649994  244.073593   
2     2017-07-27  294.700012  295.399994  286.750000  288.549988  239.834579   
3     2017-07-28  294.299988  294.750000  287.149994  291.549988  242.328125   
4     2017-07-31  289.500000  290.299988  284.500000  285.250000  237.091721   
...          ...         ...         ...         ...         ...         ...   
1231  2022-07-18  295.000000  296.299988  293.750000  295.299988  295.299988   
1232  2022-07-19  295.000000  295.600006  292.700012  294.600006  294.600006   
1233  2022-07-20  294.899994  299.500000  293.600006  298.200012  298.200012   
1234  2022-07-21  299.000000  302.200012  298.000000  299.549988  299.549988   
1235  2022-07-22  300.000000  302.500000  298.100006  300.549988  300.549988   

     

In [9]:
cls = data[['Close']]


In [10]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1, subplot_titles=('Date', 'Close'),
                    row_heights=[0.2,0.7]
                   )

fig.add_trace(go.Candlestick(x= data.index, open=data['Open'], high=data['High'], low = data['Low'], close=data['Close'], name='OHLC'))




In [11]:
trace1 = go.Scatter(
                    x = data.index,
                    y = data.Close,
                    mode = "lines",
                    name = "Close",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    )

d1 = [trace1]
layout = dict(title = 'Close values',
              xaxis= dict(title= 'Close',ticklen= 5,zeroline= False)
             )
fig = dict(data = d1, layout = layout)
iplot(fig)


In [12]:
ds = cls.values
ds

array([[291.399994],
       [293.649994],
       [288.549988],
       ...,
       [298.200012],
       [299.549988],
       [300.549988]])

In [13]:
#normalizing the data
normalizer = MinMaxScaler(feature_range=(0,1))
ds_scaled = normalizer.fit_transform(np.array(ds).reshape(-1,1))

In [14]:
#splittimg the data into training and testing data

train_size = int(len(ds_scaled)*0.70)
test_size = len(ds_scaled) - train_size

In [15]:
train_size,test_size


(865, 371)

In [16]:
ds_train, ds_test = ds_scaled[0:train_size,:], ds_scaled[train_size:len(ds_scaled),:1]


In [17]:
#creating dataset in time series for LSTM model 
#X[100,120,140,160,180] : Y[200]
def create_ds(dataset,step):
    Xtrain, Ytrain = [], []
    for i in range(len(dataset)-step-1):
        a = dataset[i:(i+step), 0]
        Xtrain.append(a)
        Ytrain.append(dataset[i + step, 0])
    return np.array(Xtrain), np.array(Ytrain)

In [18]:
#Taking 100 days price as one record for training
time_stamp = 100
X_train, y_train = create_ds(ds_train,time_stamp)
X_test, y_test = create_ds(ds_test,time_stamp)

In [19]:
X_train.shape,y_train.shape


((764, 100), (764,))

In [20]:
X_test.shape, y_test.shape


((270, 100), (270,))

In [21]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [22]:
#Creating LSTM model using keras
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
model.add(LSTM(units=50,return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1,activation='linear'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Training model with adam optimizer and mean squared error loss function
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
12/12 [==============================] - 9s 320ms/step - loss: 0.1098 - val_loss: 0.0109
Epoch 2/100
12/12 [==============================] - 2s 199ms/step - loss: 0.0188 - val_loss: 0.0134
Epoch 3/100
12/12 [==============================] - 2s 199ms/step - loss: 0.0109 - val_loss: 0.0061
Epoch 4/100
12/12 [==============================] - 2s 199ms/step - loss: 0.0065 - val_loss: 0.0043
Epoch 5/100
12/12 [==============================] - 2s 199ms/step - loss: 0.0050 - val_loss: 0.0038
Epoch 6/100
12/12 [==============================] - 2s 198ms/step - loss: 0.0045 - val_loss: 0.0042
Epoch 7/100
12/12 [==============================] - 2s 200ms/step - loss: 0.0042 - val_loss: 0.0036
Epoch 8/100
12/12 [==============================] - 2s 201ms/step - loss: 0.0040 - val_loss: 0.0039
Epoch 9/100
12/12 [==============================] - 2s 202ms/step - loss: 0.0039 - val_loss: 0.0036
Epoch 10/100
12/12 [==============================] - 2s 200ms/step - loss: 0.0038 - val_lo

In [ ]:
loss = model.history.history['loss']
trace2 = go.Scatter(
                    x = loss,
                    
                    mode = "lines",
                    name = "loss",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)')
                    )


d2 = [trace2]
layout = dict(title = 'loss',
              xaxis= dict(title= 'loss',ticklen= 5,zeroline= False)
             )
fig = dict(data = d2, layout = layout)
iplot(fig)

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
train_predict = normalizer.inverse_transform(train_predict)
test_predict = normalizer.inverse_transform(test_predict)
invds = normalizer.inverse_transform(ds_scaled)

In [ ]:

plt.plot(normalizer.inverse_transform(ds_scaled))
plt.plot(train_predict)
plt.plot(test_predict)

In [ ]:
test = np.vstack((train_predict,test_predict))


In [ ]:

plt.figure(figsize = (10,5))

plt.plot(normalizer.inverse_transform(ds_scaled), 'g', label = 'original price')
plt.plot((test), 'b', label = 'predicted price')

plt.legend()

plt.show()

In [ ]:
len(ds_test)


In [ ]:
fut_inp = ds_test[272:]


In [ ]:
fut_inp = fut_inp.reshape(1,-1)


In [ ]:
tmp_inp = list(fut_inp)


In [ ]:
fut_inp.shape


In [ ]:
#Creating list of the last 100 data
tmp_inp = tmp_inp[0].tolist()

In [ ]:
#Predicting next 10 days price suing the current data
#It will predict in sliding window manner (algorithm) with stride 1
lst_output=[]
n_steps=100
i=0
while(i<10):
    
    if(len(tmp_inp)>100):
        fut_inp = np.array(tmp_inp[1:])
        fut_inp=fut_inp.reshape(1,-1)
        fut_inp = fut_inp.reshape((1, n_steps, 1))
        yhat = model.predict(fut_inp, verbose=0)
        tmp_inp.extend(yhat[0].tolist())
        tmp_inp = tmp_inp[1:]
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        fut_inp = fut_inp.reshape((1, n_steps,1))
        yhat = model.predict(fut_inp, verbose=0)
        tmp_inp.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())
        i=i+1
    

    
print(lst_output)

In [ ]:
scale=sklearn.preprocessing.MinMaxScaler()

len(ds_scaled)

In [ ]:
#Creating a dummy plane to plot graph one after another
plot_new=np.arange(1,101)
plot_pred=np.arange(101,111)

In [ ]:
plt.plot(plot_new, normalizer.inverse_transform(ds_scaled[1137:]))
plt.plot(plot_pred, normalizer.inverse_transform(lst_output))

In [ ]:
ds_new = ds_scaled.tolist()


In [ ]:
len(ds_new)


In [ ]:
ds_new.extend(lst_output)
plt.plot(ds_new[1136:])

In [ ]:
final_graph = normalizer.inverse_transform(ds_new).tolist()
#Plotting final results with predicted value after 10 Days
plt.plot(final_graph,)
plt.ylabel("Price")
plt.xlabel("Time")
plt.title("{0} prediction of next 10d Close".format(stock_symbol))
plt.axhline(y=final_graph[len(final_graph)-1], color = 'red', linestyle = ':', label = 'NEXT 10D: {0}'.format(round(float(*final_graph[len(final_graph)-1]),2)))
plt.legend()

In [ ]:
forecast = normalizer.inverse_transform(lst_output)
print(forecast)

In [ ]:
model.save('Keras.h8')